In [8]:
import pandas as pd
import numpy as np
from nltk import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn import svm
import math
import re


fake, non_fake  = pd.read_excel('cleaned_files/Fakeset_cleaned_165.xlsx'), pd.read_excel('cleaned_files/Non-fakeset_cleaned_318.xlsx')

dataset = pd.concat([fake, non_fake])

pravda_unian_set = pd.read_excel('cleaned_files/Non-fakeset.xlsx')

testing_set = pravda_unian_set[(pravda_unian_set['url'].isin(dataset['url']) == False)]

dataset.reset_index(np.linspace(1,len(dataset)), inplace=True)


#Loading ZIK news 
challenge_set = pd.read_excel('cleaned_files/zik_news.xls')

# Instantiating word vectorizer + stopwords

In [5]:
stemmer = SnowballStemmer(language='russian')

analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
    return (stemmer.stem(w) for w in analyzer(doc) if w not in rus_stopwords)

rus_stopwords = ["c","а","алло","без","белый","близко","более","больше","большой","будем","будет","будете","будешь","будто","буду","будут","будь","бы","бывает","бывь","был","была","были","было","быть","в","важная","важное","важные","важный","вам","вами","вас","ваш","ваша","ваше","ваши","вверх","вдали","вдруг","ведь","везде","вернуться","весь","вечер","взгляд","взять","вид","видел","видеть","вместе","вне","вниз","внизу","во","вода","война","вокруг","вон","вообще","вопрос","восемнадцатый","восемнадцать","восемь","восьмой","вот","впрочем","времени","время","все","все еще","всегда","всего","всем","всеми","всему","всех","всею","всю","всюду","вся","всё","второй","вы","выйти","г","где","главный","глаз","говорил","говорит","говорить","год","года","году","голова","голос","город","да","давать","давно","даже","далекий","далеко","дальше","даром","дать","два","двадцатый","двадцать","две","двенадцатый","двенадцать","дверь","двух","девятнадцатый","девятнадцать","девятый","девять","действительно","дел","делал","делать","делаю","дело","день","деньги","десятый","десять","для","до","довольно","долго","должен","должно","должный","дом","дорога","друг","другая","другие","других","друго","другое","другой","думать","душа","е","его","ее","ей","ему","если","есть","еще","ещё","ею","её","ж","ждать","же","жена","женщина","жизнь","жить","за","занят","занята","занято","заняты","затем","зато","зачем","здесь","земля","знать","значит","значить","и","иди","идти","из","или","им","имеет","имел","именно","иметь","ими","имя","иногда","их","к","каждая","каждое","каждые","каждый","кажется","казаться","как","какая","какой","кем","книга","когда","кого","ком","комната","кому","конец","конечно","которая","которого","которой","которые","который","которых","кроме","кругом","кто","куда","лежать","лет","ли","лицо","лишь","лучше","любить","люди","м","маленький","мало","мать","машина","между","меля","менее","меньше","меня","место","миллионов","мимо","минута","мир","мира","мне","много","многочисленная","многочисленное","многочисленные","многочисленный","мной","мною","мог","могу","могут","мож","может","может быть","можно","можхо","мои","мой","мор","москва","мочь","моя","моё","мы","на","наверху","над","надо","назад","наиболее","найти","наконец","нам","нами","народ","нас","начала","начать","наш","наша","наше","наши","не","него","недавно","недалеко","нее","ней","некоторый","нельзя","нем","немного","нему","непрерывно","нередко","несколько","нет","нею","неё","ни","нибудь","ниже","низко","никакой","никогда","никто","никуда","ним","ними","них","ничего","ничто","но","новый","нога","ночь","ну","нужно","нужный","нх","о","об","оба","обычно","один","одиннадцатый","одиннадцать","однажды","однако","одного","одной","оказаться","окно","около","он","она","они","оно","опять","особенно","остаться","от","ответить","отец","откуда","отовсюду","отсюда","очень","первый","перед","писать","плечо","по","под","подойди","подумать","пожалуйста","позже","пойти","пока","пол","получить","помнить","понимать","понять","пор","пора","после","последний","посмотреть","посреди","потом","потому","почему","почти","правда","прекрасно","при","про","просто","против","процентов","путь","пятнадцатый","пятнадцать","пятый","пять","работа","работать","раз","разве","рано","раньше","ребенок","решить","россия","рука","русский","ряд","рядом","с","с кем","сам","сама","сами","самим","самими","самих","само","самого","самой","самом","самому","саму","самый","свет","свое","своего","своей","свои","своих","свой","свою","сделать","сеаой","себе","себя","сегодня","седьмой","сейчас","семнадцатый","семнадцать","семь","сидеть","сила","сих","сказал","сказала","сказать","сколько","слишком","слово","случай","смотреть","сначала","снова","со","собой","собою","советский","совсем","спасибо","спросить","сразу","стал","старый","стать","стол","сторона","стоять","страна","суть","считать","т","та","так","такая","также","таки","такие","такое","такой","там","твои","твой","твоя","твоё","те","тебе","тебя","тем","теми","теперь","тех","то","тобой","тобою","товарищ","тогда","того","тоже","только","том","тому","тот","тою","третий","три","тринадцатый","тринадцать","ту","туда","тут","ты","тысяч","у","увидеть","уж","уже","улица","уметь","утро","хороший","хорошо","хотел бы","хотеть","хоть","хотя","хочешь","час","часто","часть","чаще","чего","человек","чем","чему","через","четвертый","четыре","четырнадцатый","четырнадцать","что","чтоб","чтобы","чуть","шестнадцатый","шестнадцать","шестой","шесть","эта","эти","этим","этими","этих","это","этого","этой","этом","этому","этот","эту","я","являюсь"]

# Testing

In [6]:
X_train, X_test, y_train, y_test = train_test_split(dataset['ru_text'], 
                                                    dataset['target'], 
                                                    random_state=0)

vect = CountVectorizer(min_df=5, ngram_range=(1,3), analyzer=stemmed_words).fit(X_train)

X_train_vectorized = vect.transform(X_train)

model = svm.SVC(kernel='linear').fit(X_train_vectorized, y_train)

matrix = vect.transform(challenge_set['text'])

res = model.predict(matrix)

print({'zik_set_accuracy': (1-np.count_nonzero(res)/len(res))})

{'zik_set_accuracy': 0.6111111111111112}


# With propaganda-sensitive words 

In [12]:
pol_words = ['донецкая народная республика','ополченцы', 'фашисты', 'радикальной', 'незалежная', 'киевские власти', 'народный', 'савченко', 'народная', 'каратели', 'киев', 'ордо','террористы', 'боевики', 'гибридная', 'ато', 'незаконные вооруженные формирования', 'пророссийские боевики','бандформирования']

def political_words(row):
    data = row['text'].lower()
    words = pol_words
    for word in words:
        if word in data:
            row[word] = 1
        else:
            row[word] = 0
    return row

def build_log_scale(row):
    scale = pow(len(row['text']), 1/10)
    spmarks_ratio = len(re.findall('[,.?!:;"]', row['text']))
    score = math.log(spmarks_ratio, scale)
    return score

def build_digit_scale(row):
    scale = pow(len(row['text']), 1/10)
    digit_ratio = len(re.findall('(\d{1,2})+', row['text']))
    if digit_ratio == 0:
        score = 1
    else:
        score = math.log(digit_ratio, scale)
    return score

dataset = dataset.apply(political_words, axis=1)
dataset['special_marks'] = dataset.apply(build_log_scale, axis=1)
dataset['digit_freq'] = dataset.apply(build_digit_scale, axis=1)

challenge = challenge_set.apply(political_words, axis=1)
challenge['special_marks'] = challenge.apply(build_log_scale, axis=1)
challenge['digit_freq'] = challenge.apply(build_digit_scale, axis=1)

X_train, X_test, y_train, y_test = train_test_split(dataset.loc[:,'донецкая народная республика': 'digit_freq'], 
                                                    dataset['target'], 
                                                    random_state=0)

model = svm.SVC(kernel='linear', C=100).fit(X_train, y_train)

challenge = challenge.loc[:,'донецкая народная республика':'digit_freq']

res = model.predict(challenge.loc[:,'донецкая народная республика':'digit_freq'])

print({'zik_set_accuracy': (1-np.count_nonzero(res)/len(res))})



{'zik_set_accuracy': 0.8888888888888888}


# TF-matrix with propaganda-sensitive words 

In [20]:
from scipy.sparse import csr_matrix, hstack
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

X_train, X_test, y_train, y_test = train_test_split(dataset.loc[:,'ru_text':'бандформирования'], 
                                                    dataset['target'], 
                                                    random_state=0)

pol_words_matrix = csr_matrix(X_train.loc[:, 'донецкая народная республика': 'бандформирования'])

vect = CountVectorizer(min_df=5, ngram_range=(1,3), analyzer=stemmed_words).fit(X_train['ru_text'])

X_train_vectorized = vect.transform(X_train['ru_text'])

X_train_final = hstack([X_train_vectorized, pol_words_matrix])

model = svm.SVC(kernel='linear', C=1).fit(X_train_final, y_train)

matrix = vect.transform(challenge_set['text'])

pol_words_matrix = csr_matrix(challenge.loc[:,'донецкая народная республика':'бандформирования'])

matrix_add = hstack([matrix, pol_words_matrix])

res = model.predict(matrix_add)

print({'zik_set_accuracy': (1-np.count_nonzero(res)/len(res))})

{'zik_set_accuracy': 0.6111111111111112}
